<a href="https://colab.research.google.com/github/daiyos/finance_getdata/blob/main/finance_getdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoo-finance-api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3731 sha256=4c6a171a3460aa256994bc2b7f8d09f0cab39b3d2fa13030c0a72880abd9e466
  Stored in directory: /root/.cache/pip/wheels/df/83/e1/df62dc96545acda58900be732f6eca5f34c84f28c9bd882f75
Successfully built yahoo-finance-api2


In [3]:
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as datetime
import pandas as pd

def get_data(code):

  stock = share.Share(code)
  symbol_data = None

  try:
      # get_historical(period_type, period, frequency_type, frequency)
      symbol_data = stock.get_historical(share.PERIOD_TYPE_DAY, 10, share.FREQUENCY_TYPE_DAY, 1)
  except YahooFinanceError as e:
      print(f">>> [{code}] : {e.message}")
      return None

  # 日付処理のためにDataframeに変換
  df_symbol_data = pd.DataFrame(symbol_data)
  # タイムスタンプを変換
  df_symbol_data["datetime"] = pd.to_datetime(df_symbol_data.timestamp, unit="ms")
  # 日本時間へ変換
  df_symbol_data["datetime_jst"] = df_symbol_data["datetime"] + datetime.timedelta(hours=9)
  # 後でキーに使いやすいように日本時間を文字列に変換
  df_symbol_data["datetime_jst_str"] = df_symbol_data["datetime_jst"].apply(lambda x: x.strftime('%Y%m%d'))

  return df_symbol_data

# 日本株については.Tをつける：code + ".T"
stockList = ["AAPL","AMZN","FB","MSFT","sdasd","GOOGL","4661.T"]

# とりあえずの格納先：'code_日付'をキーとし、終値を格納する
# 取得できなかった場合はSKIPする
dic_stock = {}
for code in stockList:
  res = get_data(code)
  if(res is None):
    print(f">>> [{code}] : NoData : SKIP")
    continue

  print(code)
  # 終値
  num = len(res)
  for index in range(len(res)):
    dic_target = res[index:index+1].to_dict()
    dic_stock[code + "_" + dic_target["datetime_jst_str"][index]] = dic_target["close"][index]

print(dic_stock)


AAPL
AMZN
>>> [FB] : Not Found: No data found, symbol may be delisted
>>> [FB] : NoData : SKIP
MSFT
>>> [sdasd] : Not Found: No data found, symbol may be delisted
>>> [sdasd] : NoData : SKIP
GOOGL
4661.T
{'AAPL_20220901': 157.9600067138672, 'AAPL_20220902': 155.80999755859375, 'AAPL_20220906': 154.52999877929688, 'AAPL_20220907': 155.9600067138672, 'AAPL_20220908': 154.4600067138672, 'AAPL_20220909': 157.3699951171875, 'AMZN_20220901': 127.81999969482422, 'AMZN_20220902': 127.51000213623047, 'AMZN_20220906': 126.11000061035156, 'AMZN_20220907': 129.47999572753906, 'AMZN_20220908': 129.82000732421875, 'AMZN_20220909': 133.27000427246094, 'MSFT_20220901': 260.3999938964844, 'MSFT_20220902': 256.05999755859375, 'MSFT_20220906': 253.25, 'MSFT_20220907': 258.0899963378906, 'MSFT_20220908': 258.5199890136719, 'MSFT_20220909': 264.4599914550781, 'GOOGL_20220901': 109.73999786376953, 'GOOGL_20220902': 107.8499984741211, 'GOOGL_20220906': 106.80999755859375, 'GOOGL_20220907': 109.44999694824219